In [1]:
import sys
sys.path.insert(1, '../scr')
from PhraseDictionary import PhraseDictionary
from toolbox import Configurator, AnkiCommunicator, AnkiCardWriter
from DictionaryReader import CollinsReader, PhraseReader
from ExerciseWriter import ExerciseWriter
from Exercise import ExerciseFactory, ExerciseGatherer, ExampleSentences
import pyperclip
from pathlib import Path
import shutil
import json


def copy_folder(source_folder, destination_folder) -> None:
    destination_path = Path(destination_folder)
    source_path = Path(source_folder)
    if not destination_path.exists():
        shutil.copytree(source_path, destination_path)

### Initial set-up

In [2]:
# User inputs
date = 20240729


# Make a new folder
template_folder = '../scr/resources/Templates'
export_folder = f'./Exports/{date}'
copy_folder(template_folder, export_folder)

# Paths of the templates
def_template_path = '/Definition/template.tex'
def_completion_template_path = '/Definition_Completion/template.tex'
reading_template_path = '/Reading/template.tex'
gap_filling_template_path = '/Fill-in-the-gap/template.tex'
paraphrase_template_path = '/Paraphrase/template.tex'
dialogue_template_path = '/Dialogue/template.tex'

# Paths of the output folder
def_output_folder = export_folder + '/Definition'
def_completion_output_folder = export_folder + '/Definition_completion'
reading_output_folder = export_folder + '/Reading'
gap_filling_output_folder = export_folder + '/Fill-in-the-gap'
paraphrase_output_folder = export_folder + '/Paraphrase'
dialogue_output_folder = export_folder + '/Dialogue'

# Creation of writers
factory = ExerciseFactory()
def_gatherer = ExerciseGatherer()
def_writer = ExerciseWriter(gatherer=def_gatherer, template_path=def_template_path, output_folder=def_output_folder)
def_completion_gatherer = ExerciseGatherer()
def_completion_writer = ExerciseWriter(gatherer=def_completion_gatherer, template_path=def_completion_template_path, output_folder=def_completion_output_folder)
reading_gatherer = ExerciseGatherer()
reading_exercise_writer = ExerciseWriter(gatherer=reading_gatherer, template_path=reading_template_path, output_folder=reading_output_folder)
gap_filling_gatherer = ExerciseGatherer()
gap_filling_exercise_writer = ExerciseWriter(gatherer=gap_filling_gatherer, template_path=gap_filling_template_path, output_folder=gap_filling_output_folder)
paraphrase_gatherer = ExerciseGatherer()
paraphrase_exercise_writer = ExerciseWriter(gatherer=paraphrase_gatherer, template_path=paraphrase_template_path, output_folder=paraphrase_output_folder)
dialogue_gatherer = ExerciseGatherer()
dialogue_exercise_writer = ExerciseWriter(gatherer=dialogue_gatherer, template_path=dialogue_template_path, output_folder=dialogue_output_folder)

### Configuration for `Vocab Explorer`

In [3]:
category = 'Vocab Explorer'
# category = 'Vocab Builder'
num_of_words_to_learn = 30

configurator = Configurator(json_path=f'../scr/config/{category}/config.json')
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
reader = CollinsReader(word_list=tomorrow_new)
entry_list = reader.get_word_entry_list()
writer = AnkiCardWriter(entry_list)
Path(f'Exports/{date}_{category}').mkdir(parents=True, exist_ok=True)
writer.write_cards(f'Exports/{date}_{category}/Anki Cards.csv')
print('Go to Anki and import the cards from the file Anki Cards.csv')
configurator.study_n_words(num_of_words_to_learn)

Go to Anki and import the cards from the file Anki Cards.csv


In [5]:
anki = AnkiCommunicator()
tomorrow_new_with_def = reader.get_concise_dictionary()
tomorrow_review_with_def = anki.get_words_for_tomorrow(deck_name=category, new=False)
tomorrow_review = list(tomorrow_review_with_def.keys())
print(f'''
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')


Number of new words: 30;
Number of old words: 46.



### Configuration for `Everyday English`

In [28]:
category = 'Everyday English'
num_of_words_to_learn = 0

configurator = Configurator(json_path=f'../scr/config/{category}/config.json')
tomorrow_new = configurator.get_n_words_to_learn(num_of_words_to_learn)
reader = PhraseReader(word_list=tomorrow_new)
entry_list = reader.get_word_entry_list()
writer = AnkiCardWriter(entry_list)
Path(f'Exports/{date}_{category}').mkdir(parents=True, exist_ok=True)
writer.write_cards(f'Exports/{date}_{category}/Anki Cards.csv')
print('Go to Anki and import the cards from the file Anki Cards.csv')
configurator.study_n_words(num_of_words_to_learn)

Go to Anki and import the cards from the file Anki Cards.csv


In [31]:
anki = AnkiCommunicator()
tomorrow_new_with_def = reader.get_concise_dictionary()
tomorrow_review_with_def = anki.get_words_for_tomorrow(deck_name=category, new=False)
tomorrow_review = list(tomorrow_review_with_def.keys())
print(f'''
Number of new words: {len(tomorrow_new)};
Number of old words: {len(tomorrow_review)}.
''')


Number of new words: 0;
Number of old words: 7.



### 1. Definition

#### Set the word list and the set index here and get the prompt to create definitions

In [8]:
def_start_idx = 21
def_end_idx = 30
def_set_index = 3

def_word_list = tomorrow_new[def_start_idx-1:def_end_idx]
definitions_set = factory.create_exercise(exercise_type='Definition', word_entries={word: [""] for word in def_word_list})
definitions_set.import_definition_from_dictionary()
definitions_set.finish_import()
def_gatherer.import_exercise(definitions_set)
def_writer.render_template(set_index=def_set_index)

### 2. Definition and Example Completion

#### Enter the index

In [ ]:
def_completion_start_idx = 37
def_completion_end_idx = 48
def_completion_set_index = 4

def_completion_word_list = tomorrow_review[def_completion_start_idx-1:def_completion_end_idx]
definition_completion_exercise = factory.create_exercise(exercise_type='Definition and Example Completion', 
                                                         word_entries={term: tomorrow_review_with_def[term] for term in def_completion_word_list})
definition_completion_exercise.generate_exercise()
definition_completion_exercise.finish_import()
def_completion_gatherer.import_exercise(definition_completion_exercise)
def_completion_writer.render_template(def_completion_set_index)

### 3. Reading

#### Enter the set index and the word list here

In [ ]:
reading_start_idx = 11
reading_end_idx = 15
reading_set_index = 1
reading_word_list = tomorrow_review[reading_start_idx-1:reading_end_idx]
reading_exercise = factory.create_exercise(exercise_type='Reading', word_entries={term: tomorrow_review_with_def[term] for term in reading_word_list})

#### Paste the passage here

In [ ]:
passage = pyperclip.paste()
reading_exercise.create_prompt(passage=passage)
reading_exercise.get_prompt()

#### Paste the dialogue here

In [ ]:
dialogue = pyperclip.paste()

reading_exercise.import_dialogue(dialogue=dialogue)
reading_exercise.finish_import()
reading_gatherer.import_exercise(exercise=reading_exercise)
reading_exercise_writer.render_template(set_index=reading_set_index)

### 4. Vocabulary exercises

#### Set the word list here and get the prompt to create example sentences

In [25]:
vocab_start_idx = 31
vocab_end_idx = 46

# vocab_word_list = tomorrow_new[vocab_start_idx-1:vocab_end_idx]
# vocab_word_entries = {term: tomorrow_new_with_def[term] for term in vocab_word_list}

vocab_word_list = tomorrow_review[vocab_start_idx-1:vocab_end_idx]
vocab_word_entries = {term: tomorrow_review_with_def[term] for term in vocab_word_list}


example_sentences = ExampleSentences(word_entries=vocab_word_entries)
example_sentences.create_prompt()
example_sentences.get_prompt()

#### Paste the example sentences here

In [26]:
sentences = pyperclip.paste()
example_sentences.import_sentences(text=sentences)

#### Fill in the gap

##### Enter the set index here

In [27]:
gap_filling_set_index = 6

gap_filling_exercise = factory.create_exercise(exercise_type='Fill in the gap', word_entries=example_sentences.word_entries, example_sentences=example_sentences)
gap_filling_exercise.finish_import()
gap_filling_gatherer.import_exercise(exercise=gap_filling_exercise)
gap_filling_exercise_writer.render_template(set_index=gap_filling_set_index)

#### Paraphrase

In [ ]:
pyperclip.copy(example_sentences.get_second_prompt())

##### Paste the json string here

In [ ]:
analyzed_sentences = pyperclip.paste()
example_sentences.import_sentence_analysis(text=analyzed_sentences)

##### Enter the set index here

In [ ]:
paraphrase_set_index = 2

paraphrase_exercise = factory.create_exercise(exercise_type='Paraphrase', word_entries=example_sentences.word_entries, example_sentences=example_sentences)
paraphrase_exercise.finish_import()
paraphrase_gatherer.import_exercise(exercise=paraphrase_exercise)
paraphrase_exercise_writer.render_template(set_index=paraphrase_set_index)

### 5. Dialogue

#### Enter the set index here

In [34]:
dialogue_start_idx = 1
dialogue_end_idx = 12
dialogue_set_index = 1

# dialogue_word_list = tomorrow_new[dialogue_start_idx-1:dialogue_end_idx]
dialogue_word_list = tomorrow_review[dialogue_start_idx-1:dialogue_end_idx]
dialogue_exercise = factory.create_exercise(exercise_type='Dialogue', word_entries={term: tomorrow_review_with_def[term] for term in dialogue_word_list})
dialogue_exercise.get_prompt()

In [35]:
dialogue_dict_text = pyperclip.paste()
dialogue_dict = json.loads(dialogue_dict_text)
dialogue_exercise.generate_exercise(dialogue_dict=dialogue_dict)
dialogue_exercise.finish_import()
dialogue_gatherer.import_exercise(exercise=dialogue_exercise)
dialogue_exercise_writer.render_template(set_index=dialogue_set_index)